In [87]:
import nltk
import spacy
import time
import os
import io
import numpy as np
from sltk import sltk
import transformers
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
import matplotlib.pyplot as plt
import seaborn as sns 

In [58]:
# nltk.download('punkt')

In [59]:
# !python -m spacy download xx_ent_wiki_sm

In [60]:
# load the corpus
with open("data/sin.csv", "r") as f:
    corpus = f.read()

In [ ]:
def count_chars(txt):
    result = 0
    for char in txt:
        if(char != "" or char != " "):
            result = result + 1
    return result

num_char = count_chars(corpus)
num_words = len(corpus.split(' '))

print(f'{num_char} {num_words}')

In [72]:
# NLTK tokenization
def nltk_tokenizer():
    start_time = time.time()
    nltk_tokens = nltk.word_tokenize(corpus)
    nltk_time = time.time() - start_time
    nltk_compression = len(nltk_tokens) / (num_char + num_words)
    print("NLTK: Time = {:.2f}s, Compression = {:.2f}".format(nltk_time, nltk_compression))
    return nltk_time, nltk_compression, len(nltk_tokens)

In [ ]:
nltk_time, nltk_compression, nltk_tokens = nltk_tokenizer()
print(nltk_time, nltk_compression, nltk_tokens)

In [74]:
# spaCy tokenization
def spacy_tokenizer():
    spacy_nlp = spacy.load("xx_ent_wiki_sm")
    start_time = time.time()
    spacy_nlp.max_length = 1_500_000
    spacy_doc = spacy_nlp(corpus)
    spacy_tokens = [token.text for token in spacy_doc]
    spacy_time = time.time() - start_time
    spacy_compression = len(spacy_tokens) / (num_char + num_words)
    print("spaCy: Time = {:.2f}s, Compression = {:.2f}".format(spacy_time, spacy_compression))
    return spacy_time, spacy_compression, len(spacy_tokens)

In [ ]:
spacy_time, spacy_compression, spacy_tokens = spacy_tokenizer()
print(spacy_time, spacy_compression)

In [76]:
# BERT tokenization
def bertTokenizer():
    start_time = time.time()
    # initialize
    tokenizer = BertWordPieceTokenizer(
        clean_text=True,
        handle_chinese_chars=False,
        strip_accents=False,
        lowercase=False
    )
    # and train
    tokenizer.train(files='data/sin.csv', vocab_size=30_000, min_frequency=2,
                    limit_alphabet=1000, wordpieces_prefix='##',
                    special_tokens=[
                        '[PAD', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

    # os.mkdir('./bert')

    tokenizer.save_model('./bert', 'bert')

    # tokenize
    tokenizer = BertTokenizer.from_pretrained("./bert")
    bert_tokens = tokenizer.tokenize(corpus)
    bert_time = time.time() - start_time
    bert_compression = len(bert_tokens) / (num_char + num_words)
    print("BERT: Time = {:.2f}s, Compression = {:.2f}".format(bert_time, bert_compression))
    # print(bert_tokens)
    return bert_time, bert_compression, len(bert_tokens)

In [ ]:
bert_time, bert_compression, bert_tokens = bertTokenizer()
print(bert_time, bert_compression)

In [78]:
# sltk tokenizer
def sltTokenizer():
    start_time = time.time()

    sentences = sltk.splitSentences(corpus)
    sltk_tokens = sltk.buildVocab(sentences)
    f = open("models/vocab.txt", "w")
    f.write('\n'.join(sltk_tokens))
    f.close()

    #tokenize
    # sltk_tokens = sltk.tokenize('models/vocab.txt', sentences)
    sltk_time = time.time() - start_time
    sltk_compression = len(sltk_tokens) / (num_char + num_words)
    print("SLTK: Time = {:.2f}s, Compression = {:.2f}".format(sltk_time, sltk_compression))
    return sltk_time, sltk_compression, len(sltk_tokens)

In [ ]:
sltk_time, sltk_compression, sltk_tokens = sltTokenizer()
print(sltk_time, sltk_compression)

In [ ]:
# Plot the results
sns.set_theme()
speed = [nltk_time, spacy_time, bert_time, sltk_time]
compression = [nltk_compression, spacy_compression, bert_compression, sltk_compression]
tokenizers = ["NLTK", "spaCy", "BERT", "SLTK"]

fig, ax1 = plt.subplots(figsize=(8,4))

color = "tab:orange"
ax1.set_xlabel("Tokenizer")
ax1.set_ylabel("Speed (s)", color=color)
ax1.bar(tokenizers, speed, color=color)
ax1.tick_params(axis="y", labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = "tab:blue"
ax2.set_ylabel("Compression Factor", color=color)  # we already handled the x-label with ax1
ax2.plot(tokenizers, compression, color=color, marker='o', label='Compression Factor')
ax2.legend(loc="upper right")
ax2.tick_params(axis="y", labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.show()

In [ ]:
objects = ("NLTK", "spaCy", "BERT", "SLTK")
y_pos = np.arange(len(objects))
performance = [nltk_tokens, spacy_tokens, bert_tokens, sltk_tokens]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Vacabulary Size')
plt.title('Number of Tokens Generated')

plt.show()